# Simple Sensor Data Visualization v2

**Purpose**: Load sensor data around sync events with individual time axes for manual sync event identification.

**Features**:
- Load 4 hours around sync start time (configurable)
- Each sensor has its own independent time axis
- Time shifts controlled by Sync_Parameters.yaml
- Preprocessing done before plotting
- Simple and focused approach

## 1. Configuration

In [1]:
# Configuration parameters
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import yaml
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display, clear_output

# ========== CONFIGURATION ==========
SUBJECT_ID = "OutSense-619"  # Change this to your subject
HOURS_AROUND_SYNC = 4  # Hours to load around sync start time (2 hours before, 2 hours after)
TARGET_FREQUENCY = 25  # Hz for resampling

# Paths
script_dir = os.path.dirname(os.path.abspath('.'))
project_root = os.path.dirname(script_dir)
sync_params_path = os.path.join(project_root, 'Sync_Parameters.yaml')
sync_events_path = os.path.join(project_root, 'Sync_Events_Times.csv')
config_path = os.path.join(project_root, 'config.yaml')

print(f"📋 Configuration:")
print(f"  Subject: {SUBJECT_ID}")
print(f"  Time window: ±{HOURS_AROUND_SYNC} hours around sync start")
print(f"  Target frequency: {TARGET_FREQUENCY} Hz")
print(f"  Project root: {project_root}")

📋 Configuration:
  Subject: OutSense-619
  Time window: ±4 hours around sync start
  Target frequency: 25 Hz
  Project root: /scai_data3/scratch


## 2. Load Configuration and Sync Parameters

In [2]:
# Load main configuration
with open('/scai_data3/scratch/stirnimann_r/config.yaml', 'r') as f:
    cfg = yaml.safe_load(f)

# Load sync parameters
with open('/scai_data3/scratch/stirnimann_r/Sync_Parameters.yaml', 'r') as f:
    sync_params = yaml.safe_load(f)

# Load sync events
sync_events_df = pd.read_csv('/scai_data3/scratch/stirnimann_r/Sync_Events_Times.csv')

print(f"✅ Loaded configurations:")
print(f"  Main config: {len(cfg)} sections")
print(f"  Sync parameters: {len(sync_params)} subjects")
print(f"  Sync events: {len(sync_events_df)} entries")

# Get sync start time for the subject
subject_sync = sync_events_df[sync_events_df['Subject'] == SUBJECT_ID]
if subject_sync.empty:
    raise ValueError(f"No sync events found for subject {SUBJECT_ID}")

sync_start_str = subject_sync.iloc[0]['Sync Start']
sync_end_str = subject_sync.iloc[0]['Sync End']

# Parse sync times
sync_start_time = pd.to_datetime(sync_start_str, format='%d.%m.%Y.%H.%M.%S')
sync_end_time = pd.to_datetime(sync_end_str, format='%d.%m.%Y.%H.%M.%S')

print(f"\n🎯 Sync times for {SUBJECT_ID}:")
print(f"  Sync Start: {sync_start_time}")
print(f"  Sync End: {sync_end_time}")
print(f"  Duration: {sync_end_time - sync_start_time}")

# Calculate data window
data_window_start = sync_start_time - pd.Timedelta(hours=HOURS_AROUND_SYNC//2)
data_window_end = sync_start_time + pd.Timedelta(hours=HOURS_AROUND_SYNC//2)

print(f"\n📊 Data window ({HOURS_AROUND_SYNC}h around sync start):")
print(f"  Window Start: {data_window_start}")
print(f"  Window End: {data_window_end}")
print(f"  Total Duration: {data_window_end - data_window_start}")

✅ Loaded configurations:
  Main config: 62 sections
  Sync parameters: 16 subjects
  Sync events: 7 entries

🎯 Sync times for OutSense-619:
  Sync Start: 2023-10-03 13:24:25
  Sync End: 2023-10-05 12:54:10
  Duration: 1 days 23:29:45

📊 Data window (4h around sync start):
  Window Start: 2023-10-03 11:24:25
  Window End: 2023-10-03 15:24:25
  Total Duration: 0 days 04:00:00


## 3. Load and Import Required Functions

In [3]:
# Import data loading functions from the original notebook/scripts
import sys
sys.path.append(project_root)

# Import necessary functions (you may need to adjust these based on your actual module structure)
try:
    from raw_data_processor import (
        select_data_loader,
        modify_modality_names,
        process_modality_duplicates,
        handle_missing_data_interpolation,
        correct_timestamp_drift
    )
    print("✅ Imported functions from raw_data_processor")
except ImportError as e:
    print(f"⚠️ Could not import from raw_data_processor: {e}")
    print("You may need to adjust the import paths or copy the required functions")
    
    # Define minimal data loader selection function
    def select_data_loader(sensor_name):
        """Simple data loader selector - you may need to implement based on your data structure"""
        def simple_csv_loader(subject_dir, sensor_name, sensor_settings):
            # This is a placeholder - implement based on your actual data structure
            csv_path = os.path.join(subject_dir, f"{sensor_name}.csv")
            if os.path.exists(csv_path):
                return pd.read_csv(csv_path)
            else:
                return pd.DataFrame()
        return simple_csv_loader
    
    def modify_modality_names(data, sensor_name):
        """Simple modality name modifier"""
        return sensor_name, data
    
    def process_modality_duplicates(data, sample_rate):
        """Simple duplicate processor"""
        return data.drop_duplicates()
    
    def handle_missing_data_interpolation(data, max_interp_gap_s=2, target_freq=50):
        """Simple interpolation"""
        return data.interpolate(method='linear', limit=int(max_interp_gap_s * target_freq))
    
    def correct_timestamp_drift(timestamp, t0, t1, drift_secs):
        """Simple drift correction"""
        if t0 <= timestamp <= t1:
            progress = (timestamp - t0) / (t1 - t0)
            return timestamp + (drift_secs * progress)
        return timestamp
    
    print("📝 Using simplified placeholder functions")

# Get raw data configuration
raw_data_parsing_config = cfg.get('raw_data_parsing_config', {})
raw_data_base_dir = os.path.join(project_root, cfg.get('raw_data_input_dir', 'data'))
subject_dir = os.path.join(raw_data_base_dir, SUBJECT_ID)

print(f"\n📂 Data paths:")
print(f"  Raw data dir: {raw_data_base_dir}")
print(f"  Subject dir: {subject_dir}")
print(f"  Available sensors: {list(raw_data_parsing_config.keys())}")

✅ Imported functions from raw_data_processor

📂 Data paths:
  Raw data dir: /scai_data2/scai_datasets/interim/scai-outsense/
  Subject dir: /scai_data2/scai_datasets/interim/scai-outsense/OutSense-619
  Available sensors: ['corsano_wrist_acc', 'cosinuss_ear_acc_x_acc_y_acc_z', 'mbient_imu_wc_accelerometer', 'mbient_imu_wc_gyroscope', 'vivalnk_vv330_acceleration', 'sensomative_bottom_logger', 'sensomative_back_logger', 'corsano_bioz_acc']


## 4. Load and Process Sensor Data

In [4]:
# Load and process each sensor with time shifts from Sync_Parameters.yaml
print(f"\n=== LOADING SENSOR DATA ===")
print(f"Processing sensors for subject: {SUBJECT_ID}")
print(f"Time window: {data_window_start} to {data_window_end}")

processed_sensors = {}
subject_correction_params = sync_params.get(SUBJECT_ID, {})

for sensor_name, sensor_settings in raw_data_parsing_config.items():
    print(f"\n--- Processing sensor: {sensor_name} ---")
    
    try:
        # Load raw sensor data
        loader = select_data_loader(sensor_name)
        sensor_data_raw = loader(subject_dir, sensor_name, sensor_settings)
        
        if sensor_data_raw.empty or 'time' not in sensor_data_raw.columns:
            print(f"❌ No data loaded for {sensor_name}")
            continue
        
        print(f"📊 Loaded {len(sensor_data_raw)} raw samples")
        
        # Get time correction parameters for this sensor
        sensor_corr_params = subject_correction_params.get(sensor_name, {'unit': 's'})
        time_unit = sensor_corr_params.get('unit', 's')
        shift_val = sensor_corr_params.get('shift', 0)
        
        # Apply time corrections
        time_col_num = sensor_data_raw['time'].astype(float)
        
        # Convert to seconds if needed
        if time_unit == 'ms':
            time_col_num = time_col_num / 1000.0
        
        # Apply shift correction
        if shift_val != 0:
            time_col_num = time_col_num + shift_val
            print(f"⏱️ Applied time shift: {shift_val}s")
        
        # Apply drift correction if available
        drift_params = sensor_corr_params.get('drift')
        if drift_params and all(k in drift_params for k in ['t0', 't1', 'drift_secs']):
            t0_ts = pd.Timestamp(drift_params['t0'])
            t1_ts = pd.Timestamp(drift_params['t1'])
            if not pd.isna(t0_ts) and not pd.isna(t1_ts):
                t0, t1 = t0_ts.timestamp(), t1_ts.timestamp()
                drift = drift_params['drift_secs']
                time_col_num = time_col_num.apply(correct_timestamp_drift, args=(t0, t1, drift))
                print(f"📐 Applied drift correction: {drift}s over {t1-t0:.1f}s interval")
        
        # Convert to datetime
        corrected_timestamps = pd.to_datetime(time_col_num, unit='s', errors='coerce')
        sensor_data_corrected = sensor_data_raw.drop(columns=['time']).copy()
        sensor_data_corrected['time'] = corrected_timestamps
        sensor_data_corrected.dropna(subset=['time'], inplace=True)
        
        if sensor_data_corrected.empty:
            print(f"❌ No valid data after time correction for {sensor_name}")
            continue
        
        # Filter to data window
        original_count = len(sensor_data_corrected)
        time_mask = (sensor_data_corrected['time'] >= data_window_start) & (sensor_data_corrected['time'] <= data_window_end)
        sensor_data_filtered = sensor_data_corrected[time_mask].copy()
        
        filtered_count = len(sensor_data_filtered)
        retention_pct = (filtered_count / original_count * 100) if original_count > 0 else 0
        print(f"🔍 Filtered from {original_count} to {filtered_count} samples ({retention_pct:.1f}% retained)")
        
        if sensor_data_filtered.empty:
            print(f"❌ No data in time window for {sensor_name}")
            continue
        
        # Set time as index
        sensor_data_filtered.set_index('time', inplace=True)
        sensor_data_filtered.sort_index(inplace=True)
        
        # Apply basic preprocessing
        sample_rate = sensor_settings.get('sample_rate', TARGET_FREQUENCY)
        processed_data = process_modality_duplicates(sensor_data_filtered, sample_rate)
        processed_data = handle_missing_data_interpolation(processed_data, max_interp_gap_s=2, target_freq=TARGET_FREQUENCY)
        
        # Apply column renaming
        new_name, processed_data = modify_modality_names(processed_data, sensor_name)
        
        if processed_data.empty:
            print(f"❌ No data after preprocessing for {sensor_name}")
            continue
        
        print(f"✅ Final shape: {processed_data.shape}")
        print(f"✅ Time range: {processed_data.index.min()} to {processed_data.index.max()}")
        
        processed_sensors[new_name] = processed_data
        
    except Exception as e:
        print(f"❌ Error processing sensor {sensor_name}: {e}")
        import traceback
        traceback.print_exc()

print(f"\n📈 Successfully processed {len(processed_sensors)} sensors:")
for sensor_name, data in processed_sensors.items():
    duration = data.index.max() - data.index.min()
    print(f"  📊 {sensor_name}: {len(data)} samples, duration {duration}")

if not processed_sensors:
    raise ValueError("No sensor data was successfully processed!")


=== LOADING SENSOR DATA ===
Processing sensors for subject: OutSense-619
Time window: 2023-10-03 11:24:25 to 2023-10-03 15:24:25

--- Processing sensor: corsano_wrist_acc ---
📊 Loaded 2416320 raw samples
⏱️ Applied time shift: 7199s
🔍 Filtered from 2416320 to 201888 samples (8.4% retained)
✅ Final shape: (201888, 3)
✅ Time range: 2023-10-03 12:50:25 to 2023-10-03 15:08:27.967999935

--- Processing sensor: cosinuss_ear_acc_x_acc_y_acc_z ---
📊 Loaded 3839064 raw samples
🔍 Filtered from 3839064 to 1116751 samples (29.1% retained)
✅ Final shape: (1116751, 3)
✅ Time range: 2023-10-03 11:27:44.401999950 to 2023-10-03 15:24:24.999000072

--- Processing sensor: mbient_imu_wc_accelerometer ---
📊 Loaded 8731223 raw samples
⏱️ Applied time shift: 7214s
📐 Applied drift correction: -11s over 170985.0s interval
🔍 Filtered from 8731223 to 437389 samples (5.0% retained)
✅ Final shape: (437389, 3)
✅ Time range: 2023-10-03 12:59:02.941925049 to 2023-10-03 15:24:24.997771740

--- Processing sensor: mbie

## 5. Interactive Plotting with Independent Time Axes

In [ ]:
# Create interactive plotting tool with independent time axes
print("=== INTERACTIVE SENSOR VISUALIZATION ===")
print("🎯 Each sensor has its own independent time axis")
print("🔍 Perfect for manual sync event identification")

# Create controls
sensor_names = list(processed_sensors.keys())

# Sensor selection
sensor_selection = widgets.SelectMultiple(
    options=sensor_names,
    value=sensor_names[:3] if len(sensor_names) >= 3 else sensor_names,  # Select first 3 by default
    description='Select Sensors:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(height='200px', width='300px')
)

# Time window controls
center_time_text = widgets.Text(
    value=sync_start_time.strftime('%Y-%m-%d %H:%M:%S'),
    description='Center Time:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

window_minutes = widgets.IntSlider(
    value=60,  # 1 hour window
    min=1,
    max=240,  # 4 hours max
    step=1,
    description='Window (min):',
    style={'description_width': 'initial'}
)

# Quick jump buttons
jump_sync_start = widgets.Button(description='🎯 Jump to Sync Start', button_style='success')
jump_sync_end = widgets.Button(description='🎯 Jump to Sync End', button_style='warning')
jump_data_start = widgets.Button(description='📊 Jump to Data Start', button_style='info')
jump_data_end = widgets.Button(description='📊 Jump to Data End', button_style='info')

# Plot button
plot_button = widgets.Button(description='📈 Plot Sensors', button_style='primary', layout=widgets.Layout(width='150px'))

# Output area
plot_output = widgets.Output()

def get_center_time():
    """Get center time from text widget"""
    try:
        return pd.to_datetime(center_time_text.value)
    except:
        return sync_start_time

def update_center_time(new_time):
    """Update center time text widget"""
    center_time_text.value = new_time.strftime('%Y-%m-%d %H:%M:%S')

def plot_sensors(btn):
    """Plot selected sensors with independent time axes"""
    with plot_output:
        clear_output(wait=True)
        
        try:
            selected_sensors = list(sensor_selection.value)
            if not selected_sensors:
                print("❌ Please select at least one sensor")
                return
            
            center_time = get_center_time()
            window_mins = window_minutes.value
            
            # Calculate time window
            half_window = pd.Timedelta(minutes=window_mins/2)
            plot_start = center_time - half_window
            plot_end = center_time + half_window
            
            print(f"📊 Plotting {len(selected_sensors)} sensors")
            print(f"⏱️ Time window: {plot_start} to {plot_end} ({window_mins} minutes)")
            print(f"🎯 Center time: {center_time}")
            
            # Create plot with INDEPENDENT time axes for each sensor
            fig, axes = plt.subplots(len(selected_sensors), 1, 
                                   figsize=(16, 3*len(selected_sensors)), 
                                   sharex=False)  # Independent time axes!
            if len(selected_sensors) == 1:
                axes = [axes]
            
            for i, sensor_name in enumerate(selected_sensors):
                ax = axes[i]
                
                if sensor_name not in processed_sensors:
                    ax.text(0.5, 0.5, f'No data for {sensor_name}', 
                           ha='center', va='center', transform=ax.transAxes)
                    ax.set_title(f'{sensor_name} - No Data')
                    continue
                
                sensor_data = processed_sensors[sensor_name]
                
                # Filter to plot window
                mask = (sensor_data.index >= plot_start) & (sensor_data.index <= plot_end)
                plot_data = sensor_data[mask]
                
                if plot_data.empty:
                    ax.text(0.5, 0.5, f'No data in time window for {sensor_name}', 
                           ha='center', va='center', transform=ax.transAxes)
                    ax.set_title(f'{sensor_name} - No Data in Window')
                    continue
                
                # Plot all numeric columns
                numeric_cols = plot_data.select_dtypes(include=[np.number]).columns
                for col in numeric_cols:
                    ax.plot(plot_data.index, plot_data[col], 
                           label=col, alpha=0.7, linewidth=1)
                
                # Mark sync events
                if plot_start <= sync_start_time <= plot_end:
                    ax.axvline(sync_start_time, color='red', linestyle='--', 
                             linewidth=2, alpha=0.8, label='🎯 Sync Start')
                
                if plot_start <= sync_end_time <= plot_end:
                    ax.axvline(sync_end_time, color='darkred', linestyle='--', 
                             linewidth=2, alpha=0.8, label='🎯 Sync End')
                
                # Mark center time
                ax.axvline(center_time, color='green', linestyle=':', 
                         linewidth=1, alpha=0.6, label='Center')
                
                # Mark data boundaries for this sensor
                sensor_start = sensor_data.index.min()
                sensor_end = sensor_data.index.max()
                
                if plot_start <= sensor_start <= plot_end:
                    ax.axvline(sensor_start, color='blue', linestyle='-', 
                             linewidth=1, alpha=0.4, label='Data Start')
                
                if plot_start <= sensor_end <= plot_end:
                    ax.axvline(sensor_end, color='orange', linestyle='-', 
                             linewidth=1, alpha=0.4, label='Data End')
                
                # Formatting for EACH sensor's independent time axis
                ax.set_title(f'{sensor_name} ({len(numeric_cols)} channels)')
                ax.set_ylabel('Value')
                ax.set_xlabel('Time')
                ax.grid(True, alpha=0.3)
                
                # Format time axis for THIS sensor
                ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
                ax.xaxis.set_major_locator(mdates.MinuteLocator(interval=max(1, window_mins//10)))
                plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)
                
                # Legend if not too many columns
                if len(numeric_cols) <= 6:
                    ax.legend(bbox_to_anchor=(1.02, 1), loc='upper left', fontsize=8)
                
                print(f"  📈 {sensor_name}: {len(plot_data)} samples in window")
            
            plt.suptitle(f'Sensor Data - Independent Time Axes\n'
                        f'Window: {plot_start} to {plot_end}', 
                        fontsize=14, y=0.98)
            
            plt.tight_layout()
            plt.subplots_adjust(right=0.85, top=0.92)
            plt.show()
            
            # Show sync event info
            print(f"\n🎯 Sync Event Information:")
            print(f"  📍 Sync Start: {sync_start_time}")
            print(f"  📍 Sync End: {sync_end_time}")
            print(f"  ⏱️ Sync Duration: {sync_end_time - sync_start_time}")
            
            if plot_start <= sync_start_time <= plot_end:
                print(f"  ✅ Sync Start is visible in current window")
            else:
                print(f"  ❌ Sync Start is outside current window")
                
            if plot_start <= sync_end_time <= plot_end:
                print(f"  ✅ Sync End is visible in current window")
            else:
                print(f"  ❌ Sync End is outside current window")
            
        except Exception as e:
            print(f"❌ Error creating plot: {e}")
            import traceback
            traceback.print_exc()

# Button functions
def jump_to_sync_start(btn):
    update_center_time(sync_start_time)
    plot_sensors(None)

def jump_to_sync_end(btn):
    update_center_time(sync_end_time)
    plot_sensors(None)

def jump_to_data_start(btn):
    all_starts = [data.index.min() for data in processed_sensors.values()]
    earliest = min(all_starts)
    update_center_time(earliest + pd.Timedelta(minutes=window_minutes.value/2))
    plot_sensors(None)

def jump_to_data_end(btn):
    all_ends = [data.index.max() for data in processed_sensors.values()]
    latest = max(all_ends)
    update_center_time(latest - pd.Timedelta(minutes=window_minutes.value/2))
    plot_sensors(None)

# Connect buttons
plot_button.on_click(plot_sensors)
jump_sync_start.on_click(jump_to_sync_start)
jump_sync_end.on_click(jump_to_sync_end)
jump_data_start.on_click(jump_to_data_start)
jump_data_end.on_click(jump_to_data_end)

# Layout
controls = widgets.VBox([
    widgets.HTML("<h3>🎛️ Controls</h3>"),
    sensor_selection,
    center_time_text,
    window_minutes,
    widgets.HBox([jump_sync_start, jump_sync_end]),
    widgets.HBox([jump_data_start, jump_data_end]),
    plot_button
])

display(widgets.VBox([controls, plot_output]))

print("\n🚀 Interactive visualization ready!")
print("\n📝 Instructions:")
print("  1. Select sensors to visualize")
print("  2. Set center time and window size")
print("  3. Use quick jump buttons to navigate")
print("  4. Each sensor has its own independent time axis")
print("  5. Look for sync events marked with red dashed lines")
print("\n💡 Key Features:")
print("  ✅ Independent time axes per sensor")
print("  ✅ Time shifts from Sync_Parameters.yaml applied")
print("  ✅ Preprocessing completed before plotting")
print("  ✅ Perfect for manual sync event identification")

=== INTERACTIVE SENSOR VISUALIZATION ===
🎯 Each sensor has its own independent time axis
🔍 Perfect for manual sync event identification



🚀 Interactive visualization ready!

📝 Instructions:
  1. Select sensors to visualize
  2. Set center time and window size
  3. Use quick jump buttons to navigate
  4. Each sensor has its own independent time axis
  5. Look for sync events marked with red dashed lines

💡 Key Features:
  ✅ Independent time axes per sensor
  ✅ Time shifts from Sync_Parameters.yaml applied
  ✅ Preprocessing completed before plotting
  ✅ Perfect for manual sync event identification


## 6. Summary Information

In [6]:
# Display summary information
print("=== SUMMARY ===")
print(f"Subject: {SUBJECT_ID}")
print(f"Data window: {HOURS_AROUND_SYNC}h around sync start")
print(f"Sync start: {sync_start_time}")
print(f"Sync end: {sync_end_time}")
print(f"Processed sensors: {len(processed_sensors)}")

print("\n📊 Sensor Details:")
for sensor_name, data in processed_sensors.items():
    # Get time shift applied
    original_sensor_name = sensor_name  # May be modified by modify_modality_names
    for orig_name in raw_data_parsing_config.keys():
        if orig_name in sensor_name:
            original_sensor_name = orig_name
            break
    
    sensor_corr_params = subject_correction_params.get(original_sensor_name, {})
    shift_applied = sensor_corr_params.get('shift', 0)
    
    print(f"  📈 {sensor_name}:")
    print(f"    Samples: {len(data)}")
    print(f"    Time range: {data.index.min()} to {data.index.max()}")
    print(f"    Duration: {data.index.max() - data.index.min()}")
    print(f"    Columns: {list(data.columns)}")
    print(f"    Time shift applied: {shift_applied}s")

print("\n🎯 Ready for manual sync event identification!")
print("Use the interactive plot above to examine each sensor independently.")

=== SUMMARY ===
Subject: OutSense-619
Data window: 4h around sync start
Sync start: 2023-10-03 13:24:25
Sync end: 2023-10-05 12:54:10
Processed sensors: 8

📊 Sensor Details:
  📈 corsano_wrist:
    Samples: 201888
    Time range: 2023-10-03 12:50:25 to 2023-10-03 15:08:27.967999935
    Duration: 0 days 02:18:02.967999935
    Columns: ['wrist_acc_x', 'wrist_acc_y', 'wrist_acc_z']
    Time shift applied: 0s
  📈 cosinuss_ear:
    Samples: 1116751
    Time range: 2023-10-03 11:27:44.401999950 to 2023-10-03 15:24:24.999000072
    Duration: 0 days 03:56:40.597000122
    Columns: ['ear_acc_x', 'ear_acc_y', 'ear_acc_z']
    Time shift applied: 0s
  📈 mbient_acc:
    Samples: 437389
    Time range: 2023-10-03 12:59:02.941925049 to 2023-10-03 15:24:24.997771740
    Duration: 0 days 02:25:22.055846691
    Columns: ['x_axis_g', 'y_axis_g', 'z_axis_g']
    Time shift applied: 0s
  📈 mbient_gyro:
    Samples: 437031
    Time range: 2023-10-03 12:59:09.867031097 to 2023-10-03 15:24:24.987225771
    Du